## Stappenplan:
1. Data transformation
- Pivot; clean; remaining features:
      ['Datum', 'Binnengekomen bij Provider', 'Doorgeleid naar keuzemenu',
       'Door beller opgehangen in menu', 'Automatisch behandeld in menu',
       'Doorverbindingen_totaal',  'Wachtrij_totaal', 'Wachtrij_opgehangen_totaal',
       'Geholpen_totaal', 'Gemiddelde_wachttijd_totaal', & per cat:
       'Doorverbindingen_CAT', 
       'Wachtrij_CAT',
       'Wachtrij_opgehangen_CAT',
       'Geholpen_CAT', 
       'Gemiddelde_wachttijd_CAT']

2. Data analysis
- Moving average of calls/waiting time per category

- Is wachttijd consistent over time? En aantal oproepen?

- Is wachttijd correlated with complexity score?

- Similarity between categories

- Effect of the deadlines on calls?

2. Data enrichment
- Calender events/holidays/tax deadlines
- Weather

In [ ]:
pip install workalendar

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import seaborn as sns
from cycler import cycler
from workalendar.europe import Netherlands
import json
from datetime import date, timedelta
from datetime import datetime 
import calendar

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

line_cycler   = (cycler(color=["#E69F00", "#56B4E9", "#009E73", "#0072B2", "#D55E00", "#CC79A7", "#F0E442"]) +
                 cycler(linestyle=["-", "--", "-.", ":", "-", "--", "-."]))

## 1. Data transformation

### Data inladen; kolom 'Datum' in data format zetten

In [ ]:
data = pd.read_csv('belastingtelefoon_odt003 (3).csv', sep = ';', decimal = ",") # douane eruit

In [ ]:
data['Datum'] = pd.to_datetime(data['Datum'], format =  "%d/%m/%Y")

### Kolommen hernoemen; null-waardes vervangen door 0

In [ ]:
data = data.fillna(0) 

In [ ]:
data.columns = ['Datum', 'Binnengekomen bij Provider', 'Doorgeleid naar keuzemenu',
       'Door beller opgehangen in menu', 'Automatisch behandeld in menu',
       'Thema', 'Doorverbindingen', 'Wachtrij',
       'Wachtrij_opgehangen', 'Geholpen',
       'Gemiddelde_wachttijd','','']
data=data[['Datum', 'Binnengekomen bij Provider', 'Doorgeleid naar keuzemenu',
       'Door beller opgehangen in menu', 'Automatisch behandeld in menu',
       'Thema', 'Doorverbindingen', 'Wachtrij',
       'Wachtrij_opgehangen', 'Geholpen',
       'Gemiddelde_wachttijd']]

In [ ]:
df_dagelijkse_totaal = data[data['Thema'] == 'TOTAAL'][['Datum', 'Binnengekomen bij Provider', 'Doorgeleid naar keuzemenu',
       'Door beller opgehangen in menu', 'Automatisch behandeld in menu']]

### Pivot de thema kolommen en join deze df met de totals dataframe

In [ ]:
df_dagelijkse_themas = data.pivot(index='Datum', columns='Thema', values=['Doorverbindingen', 'Wachtrij',
       'Wachtrij_opgehangen', 'Geholpen',
       'Gemiddelde_wachttijd'])

In [ ]:
df_totaal = df_dagelijkse_totaal.join(df_dagelijkse_themas, on = 'Datum', how = 'inner')

In [ ]:
df_totaal.columns

### Trek de douane kolommen af van het totaal (aangezien douane enkel een thema was tot februari 2020). Ga vervolgens verder met de resterende kolommen. 

In [ ]:
df_totaal = df_totaal.fillna(0)
df_totaal['Binnengekomen bij Provider'] = df_totaal['Binnengekomen bij Provider'] - df_totaal[('Doorverbindingen', 'DOUANE')]
df_totaal['Doorgeleid naar keuzemenu'] = df_totaal['Doorgeleid naar keuzemenu'] - df_totaal[('Doorverbindingen', 'DOUANE')]
df_totaal[('Doorverbindingen', 'TOTAAL')] = df_totaal[('Doorverbindingen', 'TOTAAL')] - df_totaal[('Doorverbindingen', 'DOUANE')]
df_totaal[('Wachtrij', 'TOTAAL')] = df_totaal[('Wachtrij', 'TOTAAL')] - df_totaal[('Wachtrij', 'DOUANE')]
df_totaal[('Wachtrij_opgehangen', 'TOTAAL')] = df_totaal[('Wachtrij_opgehangen', 'TOTAAL')] - df_totaal[('Wachtrij_opgehangen', 'DOUANE')]
df_totaal[('Geholpen', 'TOTAAL')] = df_totaal[('Geholpen', 'TOTAAL')] - df_totaal[('Geholpen', 'DOUANE')]

### Trek de ontbrekende kolommen af van het totaal (aangezien we geen informatie hebben over de beller, kunnen we hier niets mee). Dit zijn 13 rijen. 

In [ ]:
df_totaal['Wachtrij', 'ONTBREEKT'][df_totaal[('Wachtrij', 'ONTBREEKT')].notna()]

In [ ]:
df_totaal['Binnengekomen bij Provider'] = df_totaal['Binnengekomen bij Provider'] - df_totaal[('Doorverbindingen', 'ONTBREEKT')]
df_totaal['Doorgeleid naar keuzemenu'] = df_totaal['Doorgeleid naar keuzemenu'] - df_totaal[('Doorverbindingen', 'ONTBREEKT')]
df_totaal[('Doorverbindingen', 'TOTAAL')] = df_totaal[('Doorverbindingen', 'TOTAAL')] - df_totaal[('Doorverbindingen', 'ONTBREEKT')]
df_totaal[('Wachtrij', 'TOTAAL')] = df_totaal[('Wachtrij', 'TOTAAL')] - df_totaal[('Wachtrij', 'ONTBREEKT')]
df_totaal[('Wachtrij_opgehangen', 'TOTAAL')] = df_totaal[('Wachtrij_opgehangen', 'TOTAAL')] - df_totaal[('Wachtrij_opgehangen', 'ONTBREEKT')]
df_totaal[('Geholpen', 'TOTAAL')] = df_totaal[('Geholpen', 'TOTAAL')] - df_totaal[('Geholpen', 'ONTBREEKT')]

### Op 2022-09-14 zijn er geen oproepen geregistreerd. Het is mij onbekend of dit komt door een interne storing bij de belastingdienst, maar het lijkt erop dat die dag wel oproepen zijn binnengekomen, doorgeleid en afgewerkt, aangezien online geen informatie te vinden valt. Logischerwijs moet deze datum uit de data worden verwijderd. 

In [ ]:
df_totaal = df_totaal.drop(axis=0, index = df_totaal[df_totaal['Datum'] == '2022-09-14'].index)

In [ ]:

df_totaal = df_totaal[['Datum',
                    'Binnengekomen bij Provider',
                     'Doorgeleid naar keuzemenu',
                'Door beller opgehangen in menu',
                 'Automatisch behandeld in menu',
                    ('Doorverbindingen', 'AUTO'),
              ('Doorverbindingen', 'BUITENLAND'),
                  ('Doorverbindingen', 'DOUANE'),
           ('Doorverbindingen', 'INTERMEDIAIRS'),
           ('Doorverbindingen', 'ONDERNEMINGEN'),
               ('Doorverbindingen', 'ONTBREEKT'),
                  ('Doorverbindingen', 'OVERIG'),
           ('Doorverbindingen', 'PARTICULIEREN'),
               ('Doorverbindingen', 'TOESLAGEN'),
                  ('Doorverbindingen', 'TOTAAL'),
                            ('Wachtrij', 'AUTO'),
                      ('Wachtrij', 'BUITENLAND'),
                   ('Wachtrij', 'INTERMEDIAIRS'),
                   ('Wachtrij', 'ONDERNEMINGEN'),
                       ('Wachtrij', 'ONTBREEKT'),
                          ('Wachtrij', 'OVERIG'),
                   ('Wachtrij', 'PARTICULIEREN'),
                       ('Wachtrij', 'TOESLAGEN'),
                          ('Wachtrij', 'TOTAAL'),
                 ('Wachtrij_opgehangen', 'AUTO'),
           ('Wachtrij_opgehangen', 'BUITENLAND'),
        ('Wachtrij_opgehangen', 'INTERMEDIAIRS'),
        ('Wachtrij_opgehangen', 'ONDERNEMINGEN'),
            ('Wachtrij_opgehangen', 'ONTBREEKT'),
               ('Wachtrij_opgehangen', 'OVERIG'),
        ('Wachtrij_opgehangen', 'PARTICULIEREN'),
            ('Wachtrij_opgehangen', 'TOESLAGEN'),
               ('Wachtrij_opgehangen', 'TOTAAL'),
                            ('Geholpen', 'AUTO'),
                      ('Geholpen', 'BUITENLAND'),
                   ('Geholpen', 'INTERMEDIAIRS'),
                   ('Geholpen', 'ONDERNEMINGEN'),
                       ('Geholpen', 'ONTBREEKT'),
                          ('Geholpen', 'OVERIG'),
                   ('Geholpen', 'PARTICULIEREN'),
                       ('Geholpen', 'TOESLAGEN'),
                          ('Geholpen', 'TOTAAL'),
                ('Gemiddelde_wachttijd', 'AUTO'),
          ('Gemiddelde_wachttijd', 'BUITENLAND'),
       ('Gemiddelde_wachttijd', 'INTERMEDIAIRS'),
       ('Gemiddelde_wachttijd', 'ONDERNEMINGEN'),
           ('Gemiddelde_wachttijd', 'ONTBREEKT'),
              ('Gemiddelde_wachttijd', 'OVERIG'),
       ('Gemiddelde_wachttijd', 'PARTICULIEREN'),
           ('Gemiddelde_wachttijd', 'TOESLAGEN'),
              ('Gemiddelde_wachttijd', 'TOTAAL')]]

# Maand en jaar kolommen toevoegen
df_totaal['Maand'] = df_totaal['Datum'].dt.strftime('%b')
df_totaal['Jaar'] = df_totaal['Datum'].dt.strftime('%Y')

# Complexiteitsscores toevoegen
df_totaal['ComplexityScore_ondernemingen'] = 1
df_totaal['ComplexityScore_particulieren'] = 2
df_totaal['ComplexityScore_buitenland'] = 3
df_totaal['ComplexityScore_overig'] = 4
df_totaal['ComplexityScore_intermediairs'] = 5
df_totaal['ComplexityScore_toeslagen'] = 6
df_totaal['ComplexityScore_auto'] = 7

df_totaal['DagvdWeek'] = df_totaal['Datum'].dt.dayofweek
df_totaal['Maand'] = df_totaal['Datum'].dt.month
df_totaal['Dag'] = df_totaal['Datum'].dt.day
df_totaal['WeekNr'] = df_totaal['Datum'].dt.isocalendar().week

In [ ]:
df_totaal = df_totaal.reset_index(drop = True)

### Aparte dataframe aanmaken voor elk thema
Particulieren, toeslagen, ondernemingen, buitenland, auto, intermediairs, overig

In [ ]:
# DF Particulieren
df_particulieren = df_totaal[[ 'Datum', 'Maand', 'Jaar',
                    'DagvdWeek', 'Dag', 'WeekNr', 
                    'Binnengekomen bij Provider',
                    'Doorgeleid naar keuzemenu',
                    'Door beller opgehangen in menu',
                    'Automatisch behandeld in menu',
                    'ComplexityScore_particulieren',
                    ('Doorverbindingen', 'PARTICULIEREN'),
                    ('Doorverbindingen', 'TOTAAL'),
                    ('Wachtrij', 'PARTICULIEREN'),
                    ('Wachtrij', 'TOTAAL'),
                    ('Wachtrij_opgehangen', 'PARTICULIEREN'),
                    ('Wachtrij_opgehangen', 'TOTAAL'),
                    ('Geholpen', 'PARTICULIEREN'),
                    ('Geholpen', 'TOTAAL'),
                    ('Gemiddelde_wachttijd', 'PARTICULIEREN'),
                    ('Gemiddelde_wachttijd', 'TOTAAL')]]

df_particulieren.columns = [ 'Datum', 'Maand', 'Jaar',
                    'DagvdWeek', 'Dag', 'WeekNr', 
                    'Binnengekomen bij Provider',
                    'Doorgeleid naar keuzemenu',
                    'Door beller opgehangen in menu',
                    'Automatisch behandeld in menu',
                    'ComplexityScore_particulieren',
                    ('Doorverbindingen_particulieren'),
                    ('Doorverbindingen_totaal'),
                    ('Wachtrij_particulieren'),
                    ('Wachtrij_totaal'),
                    ('Wachtrij_opgehangen_particulieren'),
                    ('Wachtrij_opgehangen_totaal'),
                    ('Geholpen_particulieren'),
                    ('Geholpen_totaal'),
                    ('Gemiddelde_wachttijd_particulieren'),
                    ('Gemiddelde_wachttijd_totaal')]

In [ ]:
# DF toeslagen
df_toeslagen = df_totaal[[ 'Datum', 'Maand', 'Jaar',
                    'DagvdWeek', 'Dag', 'WeekNr', 
                    'Binnengekomen bij Provider',
                    'Doorgeleid naar keuzemenu',
                    'Door beller opgehangen in menu',
                    'Automatisch behandeld in menu',
                    'ComplexityScore_toeslagen',
                    ('Doorverbindingen', 'TOESLAGEN'),
                    ('Doorverbindingen', 'TOTAAL'),
                    ('Wachtrij', 'TOESLAGEN'),
                    ('Wachtrij', 'TOTAAL'),
                    ('Wachtrij_opgehangen', 'TOESLAGEN'),
                    ('Wachtrij_opgehangen', 'TOTAAL'),
                    ('Geholpen', 'TOESLAGEN'),
                    ('Geholpen', 'TOTAAL'),
                    ('Gemiddelde_wachttijd', 'TOESLAGEN'),
                    ('Gemiddelde_wachttijd', 'TOTAAL')]]

df_toeslagen.columns =  ['Datum', 'Maand', 'Jaar',
                      'DagvdWeek', 'Dag', 'WeekNr', 
                      'Binnengekomen bij Provider',
                      'Doorgeleid naar keuzemenu',
                      'Door beller opgehangen in menu',
                      'Automatisch behandeld in menu',
                      'ComplexityScore_ondernemingen',
                      'Doorverbindingen_toeslagen',
                      'Doorverbindingen_totaal',
                      'Wachtrij_toeslagen',
                      'Wachtrij_totaal',
                      'Wachtrij_opgehangen_toeslagen',
                      'Wachtrij_opgehangen_totaal',
                      'Geholpen_toeslagen',
                      'Geholpen_totaal',
                      'Gemiddelde_wachttijd_toeslagen',
                      'Gemiddelde_wachttijd_totaal']

In [ ]:
# DF ondernemingen
df_ondernemingen = df_totaal[[ 'Datum', 'Maand', 'Jaar',
                    'DagvdWeek', 'Dag', 'WeekNr', 
                    'Binnengekomen bij Provider',
                    'Doorgeleid naar keuzemenu',
                    'Door beller opgehangen in menu',
                    'Automatisch behandeld in menu',
                    'ComplexityScore_ondernemingen',
                    ('Doorverbindingen', 'ONDERNEMINGEN'),
                    ('Doorverbindingen', 'TOTAAL'),
                    ('Wachtrij', 'ONDERNEMINGEN'),
                    ('Wachtrij', 'TOTAAL'),
                    ('Wachtrij_opgehangen', 'ONDERNEMINGEN'),
                    ('Wachtrij_opgehangen', 'TOTAAL'),
                    ('Geholpen', 'ONDERNEMINGEN'),
                    ('Geholpen', 'TOTAAL'),
                    ('Gemiddelde_wachttijd', 'ONDERNEMINGEN'),
                    ('Gemiddelde_wachttijd', 'TOTAAL')]]

df_ondernemingen.columns =  ['Datum', 'Maand', 'Jaar',
                      'DagvdWeek', 'Dag', 'WeekNr', 
                      'Binnengekomen bij Provider',
                      'Doorgeleid naar keuzemenu',
                      'Door beller opgehangen in menu',
                      'Automatisch behandeld in menu',
                      'ComplexityScore_ondernemingen',
                      'Doorverbindingen_ondernemingen',
                      'Doorverbindingen_totaal',
                      'Wachtrij_ondernemingen',
                      'Wachtrij_totaal',
                      'Wachtrij_opgehangen_ondernemingen',
                      'Wachtrij_opgehangen_totaal',
                      'Geholpen_ondernemingen',
                      'Geholpen_totaal',
                      'Gemiddelde_wachttijd_ondernemingen',
                      'Gemiddelde_wachttijd_totaal']

In [ ]:
# DF buitenland
df_buitenland = df_totaal[[ 'Datum', 'Maand', 'Jaar',
                    'DagvdWeek', 'Dag', 'WeekNr', 
                    'Binnengekomen bij Provider',
                    'Doorgeleid naar keuzemenu',
                    'Door beller opgehangen in menu',
                    'Automatisch behandeld in menu',
                    'ComplexityScore_buitenland',
                    ('Doorverbindingen', 'BUITENLAND'),
                    ('Doorverbindingen', 'TOTAAL'),
                    ('Wachtrij', 'BUITENLAND'),
                    ('Wachtrij', 'TOTAAL'),
                    ('Wachtrij_opgehangen', 'BUITENLAND'),
                    ('Wachtrij_opgehangen', 'TOTAAL'),
                    ('Geholpen', 'BUITENLAND'),
                    ('Geholpen', 'TOTAAL'),
                    ('Gemiddelde_wachttijd', 'BUITENLAND'),
                    ('Gemiddelde_wachttijd', 'TOTAAL')]]

df_buitenland.columns =  ['Datum', 'Maand', 'Jaar',
                      'DagvdWeek', 'Dag', 'WeekNr', 
                      'Binnengekomen bij Provider',
                      'Doorgeleid naar keuzemenu',
                      'Door beller opgehangen in menu',
                      'Automatisch behandeld in menu',
                      'ComplexityScore_buitenland',
                      'Doorverbindingen_buitenland',
                      'Doorverbindingen_totaal',
                      'Wachtrij_buitenland',
                      'Wachtrij_totaal',
                      'Wachtrij_opgehangen_buitenland',
                      'Wachtrij_opgehangen_totaal',
                      'Geholpen_buitenland',
                      'Geholpen_totaal',
                      'Gemiddelde_wachttijd_buitenland',
                      'Gemiddelde_wachttijd_totaal']

In [ ]:
# DF auto
df_auto = df_totaal[[ 'Datum', 'Maand', 'Jaar',
                    'DagvdWeek', 'Dag', 'WeekNr', 
                    'Binnengekomen bij Provider',
                    'Doorgeleid naar keuzemenu',
                    'Door beller opgehangen in menu',
                    'Automatisch behandeld in menu',
                    'ComplexityScore_auto',
                    ('Doorverbindingen', 'AUTO'),
                    ('Doorverbindingen', 'TOTAAL'),
                    ('Wachtrij', 'AUTO'),
                    ('Wachtrij', 'TOTAAL'),
                    ('Wachtrij_opgehangen', 'AUTO'),
                    ('Wachtrij_opgehangen', 'TOTAAL'),
                    ('Geholpen', 'AUTO'),
                    ('Geholpen', 'TOTAAL'),
                    ('Gemiddelde_wachttijd', 'AUTO'),
                    ('Gemiddelde_wachttijd', 'TOTAAL')]]

df_auto.columns =  ['Datum', 'Maand', 'Jaar',
                      'DagvdWeek', 'Dag', 'WeekNr', 
                      'Binnengekomen bij Provider',
                      'Doorgeleid naar keuzemenu',
                      'Door beller opgehangen in menu',
                      'Automatisch behandeld in menu',
                      'ComplexityScore_auto',
                      'Doorverbindingen_auto',
                      'Doorverbindingen_totaal',
                      'Wachtrij_auto',
                      'Wachtrij_totaal',
                      'Wachtrij_opgehangen_auto',
                      'Wachtrij_opgehangen_totaal',
                      'Geholpen_auto',
                      'Geholpen_totaal',
                      'Gemiddelde_wachttijd_auto',
                      'Gemiddelde_wachttijd_totaal']

In [ ]:
# DF intermediairs
df_intermediairs = df_totaal[[ 'Datum', 'Maand', 'Jaar',
                    'DagvdWeek', 'Dag', 'WeekNr', 
                    'Binnengekomen bij Provider',
                    'Doorgeleid naar keuzemenu',
                    'Door beller opgehangen in menu',
                    'Automatisch behandeld in menu',
                    'ComplexityScore_intermediairs',
                    ('Doorverbindingen', 'INTERMEDIAIRS'),
                    ('Doorverbindingen', 'TOTAAL'),
                    ('Wachtrij', 'INTERMEDIAIRS'),
                    ('Wachtrij', 'TOTAAL'),
                    ('Wachtrij_opgehangen', 'INTERMEDIAIRS'),
                    ('Wachtrij_opgehangen', 'TOTAAL'),
                    ('Geholpen', 'INTERMEDIAIRS'),
                    ('Geholpen', 'TOTAAL'),
                    ('Gemiddelde_wachttijd', 'INTERMEDIAIRS'),
                    ('Gemiddelde_wachttijd', 'TOTAAL')]]

df_intermediairs.columns =  ['Datum', 'Maand', 'Jaar',
                      'DagvdWeek', 'Dag', 'WeekNr', 
                      'Binnengekomen bij Provider',
                      'Doorgeleid naar keuzemenu',
                      'Door beller opgehangen in menu',
                      'Automatisch behandeld in menu',
                      'ComplexityScore_intermediairs',
                      'Doorverbindingen_intermediairs',
                      'Doorverbindingen_totaal',
                      'Wachtrij_intermediairs',
                      'Wachtrij_totaal',
                      'Wachtrij_opgehangen_intermediairs',
                      'Wachtrij_opgehangen_totaal',
                      'Geholpen_intermediairs',
                      'Geholpen_totaal',
                      'Gemiddelde_wachttijd_intermediairs',
                      'Gemiddelde_wachttijd_totaal']

In [ ]:
# DF overig
df_overig = df_totaal[[ 'Datum', 'Maand', 'Jaar',
                    'DagvdWeek', 'Dag', 'WeekNr', 
                    'Binnengekomen bij Provider',
                    'Doorgeleid naar keuzemenu',
                    'Door beller opgehangen in menu',
                    'Automatisch behandeld in menu',
                    'ComplexityScore_overig',
                    ('Doorverbindingen', 'OVERIG'),
                    ('Doorverbindingen', 'TOTAAL'),
                    ('Wachtrij', 'OVERIG'),
                    ('Wachtrij', 'TOTAAL'),
                    ('Wachtrij_opgehangen', 'OVERIG'),
                    ('Wachtrij_opgehangen', 'TOTAAL'),
                    ('Geholpen', 'OVERIG'),
                    ('Geholpen', 'TOTAAL'),
                    ('Gemiddelde_wachttijd', 'OVERIG'),
                    ('Gemiddelde_wachttijd', 'TOTAAL')]]

df_overig.columns =  ['Datum', 'Maand', 'Jaar',
                      'DagvdWeek', 'Dag', 'WeekNr', 
                      'Binnengekomen bij Provider',
                      'Doorgeleid naar keuzemenu',
                      'Door beller opgehangen in menu',
                      'Automatisch behandeld in menu',
                      'ComplexityScore_overig',
                      'Doorverbindingen_overig',
                      'Doorverbindingen_totaal',
                      'Wachtrij_overig',
                      'Wachtrij_totaal',
                      'Wachtrij_opgehangen_overig',
                      'Wachtrij_opgehangen_totaal',
                      'Geholpen_overig',
                      'Geholpen_totaal',
                      'Gemiddelde_wachttijd_overig',
                      'Gemiddelde_wachttijd_totaal']

## 2. Data analysis

### Trend van inkomende calls jaarlijks


#### Totaal

In [ ]:
plt.rc("axes", prop_cycle=line_cycler)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2019,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2020,1,1))]\
                .set_index('Datum')[['Doorverbindingen_totaal']]\
                .plot(title="Calls totaal 2019",figsize=(12,8))
plt.xticks(rotation = 90)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2020,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2021,1,1))]\
                .set_index('Datum')[['Doorverbindingen_totaal']]\
                .plot(title="Calls totaal 2020",figsize=(12,8))
plt.xticks(rotation = 90)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2021,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2022,1,1))]\
                .set_index('Datum')[['Doorverbindingen_totaal']]\
                .plot(title="Calls totaal 2021",figsize=(12,8))
plt.xticks(rotation = 90)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2022,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2023,1,1))]\
                .set_index('Datum')[['Doorverbindingen_totaal']]\
                .plot(title="Calls totaal 2022",figsize=(12,8))
plt.xticks(rotation = 90)

plt.show()

In [ ]:
df_particulieren['Doorverbindingen_totaal'].mean()

In [ ]:
plt.figure(figsize = (20, 8))
plt.title('Calls totaal 2019/01-2022/09')
plt.xlabel('Datum')
plt.ylabel('Aantal calls')          

plt.plot(df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2019,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2020,1,1))].set_index('Datum')[['Doorverbindingen_totaal']],marker=',', label= '2019')
plt.plot(df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2020,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2021,1,1))].set_index('Datum')[['Doorverbindingen_totaal']],marker = ',',label = '2020')
plt.plot(df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2021,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2022,1,1))].set_index('Datum')[['Doorverbindingen_totaal']],marker = ',',label = '2021')
plt.plot(df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2022,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2023,1,1))].set_index('Datum')[['Doorverbindingen_totaal']],marker = ',',label = '2022')

df_particulieren['SMA_calls'] = df_particulieren['Doorverbindingen_totaal'].rolling(30).mean()
plt.plot(df_particulieren['Datum'], df_particulieren['SMA_calls'], color = 'black')
plt.legend()


In [ ]:
unieke_jaren = list(df_particulieren['Jaar'].unique())
for i, jaren in enumerate(unieke_jaren):
    fig, ax = plt.subplots()
    fig.set_size_inches((12,4))
    ax.set_ylim([0,100000])
    sns.boxplot(x = 'Maand', y = 'Doorverbindingen_totaal', data = df_particulieren[df_particulieren['Jaar'] == jaren], ax=ax)
    plt.title('Doorverbindingen totaal per maand in ' +str(unieke_jaren[i]))
    plt.show()

#### Particulieren

In [ ]:
df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2019,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2020,1,1))]\
                .set_index('Datum')[['Doorverbindingen_particulieren']]\
                .plot(title="Calls particulieren 2019",figsize=(12,8))
plt.xticks(rotation = 90)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2020,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2021,1,1))]\
                .set_index('Datum')[['Doorverbindingen_particulieren']]\
                .plot(title="Calls particulieren 2020",figsize=(12,8))
plt.xticks(rotation = 90)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2021,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2022,1,1))]\
                .set_index('Datum')[['Doorverbindingen_particulieren']]\
                .plot(title="Calls particulieren 2021",figsize=(12,8))
plt.xticks(rotation = 90)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2022,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2023,1,1))]\
                .set_index('Datum')[['Doorverbindingen_particulieren']]\
                .plot(title="Calls particulieren 2022",figsize=(12,8))
plt.xticks(rotation = 90)

plt.show()

In [ ]:
df_particulieren[df_particulieren['Jaar'] == '2021'][df_particulieren['Maand'] == 'Oct'][['Datum', 'Doorverbindingen_particulieren']]#.max()

In [ ]:
df_particulieren[df_particulieren['Jaar'] == '2022'][df_particulieren['Maand'] == 'Jun'][['Datum', 'Doorverbindingen_particulieren']]

In [ ]:
df_particulieren[df_particulieren['Jaar'] == '2021'][df_particulieren['Doorverbindingen_particulieren'] == 22858]

In [ ]:
df_particulieren['Doorverbindingen_particulieren'].mean()

In [ ]:
plt.figure(figsize = (20, 8))
plt.title('Calls particulieren 2019/01-2022/09')
plt.xlabel('Datum')
plt.ylabel('Aantal calls')
#plt.ylim(0,42000)

plt.plot(df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2019,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2020,1,1))].set_index('Datum')[['Doorverbindingen_particulieren']],marker=',', label= '2019')
plt.plot(df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2020,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2021,1,1))].set_index('Datum')[['Doorverbindingen_particulieren']],marker = ',', label = '2020')
plt.plot(df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2021,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2022,1,1))].set_index('Datum')[['Doorverbindingen_particulieren']],marker = ',', label = '2021')
plt.plot(df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2022,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2023,1,1))].set_index('Datum')[['Doorverbindingen_particulieren']],marker = ',', label = '2022')

df_particulieren['SMA_calls'] = df_particulieren['Doorverbindingen_particulieren'].rolling(30).mean()
plt.plot(df_particulieren['Datum'], df_particulieren['SMA_calls'], color = 'black')
plt.legend()

In [ ]:
unieke_jaren = list(df_particulieren['Jaar'].unique())
for i, jaren in enumerate(unieke_jaren):
    fig, ax = plt.subplots()
    fig.set_size_inches((12,4))
    ax.set_ylim([0,40000])
    sns.boxplot(x = 'Maand', y = 'Doorverbindingen_particulieren', data = df_particulieren[df_particulieren['Jaar'] == jaren], ax=ax)
    plt.title('Doorverbindingen voor particulieren per maand in ' +str(unieke_jaren[i]))
    plt.show()

#### Ondernemingen

In [ ]:
df_ondernemingen[(df_ondernemingen['Datum'] > pd.Timestamp(2019,1,1)) &(df_ondernemingen['Datum'] < pd.Timestamp(2020,1,1))]\
                .set_index('Datum')[['Doorverbindingen_ondernemingen']]\
                .plot(title="Calls ondernemingen 2019",figsize=(12,8))
plt.xticks(rotation = 90)

df_ondernemingen[(df_ondernemingen['Datum'] > pd.Timestamp(2020,1,1)) &(df_ondernemingen['Datum'] < pd.Timestamp(2021,1,1))]\
                .set_index('Datum')[['Doorverbindingen_ondernemingen']]\
                .plot(title="Calls ondernemingen 2020",figsize=(12,8))
plt.xticks(rotation = 90)

df_ondernemingen[(df_ondernemingen['Datum'] > pd.Timestamp(2021,1,1)) &(df_ondernemingen['Datum'] < pd.Timestamp(2022,1,1))]\
                .set_index('Datum')[['Doorverbindingen_ondernemingen']]\
                .plot(title="Calls ondernemingen 2021",figsize=(12,8))
plt.xticks(rotation = 90)

df_ondernemingen[(df_ondernemingen['Datum'] > pd.Timestamp(2022,1,1)) &(df_ondernemingen['Datum'] < pd.Timestamp(2023,1,1))]\
                .set_index('Datum')[['Doorverbindingen_ondernemingen']]\
                .plot(title="Calls ondernemingen 2022",figsize=(12,8))
plt.xticks(rotation = 90)

plt.show()


In [ ]:
plt.figure(figsize = (20, 8))
plt.title('Calls ondernemingen 2019/01-2022/09')
plt.xlabel('Datum')
plt.ylabel('Aantal calls')
#plt.ylim(0,42000)

plt.plot(df_ondernemingen[(df_ondernemingen['Datum'] > pd.Timestamp(2019,1,1)) &(df_ondernemingen['Datum'] < pd.Timestamp(2020,1,1))].set_index('Datum')[['Doorverbindingen_ondernemingen']],marker=',', label= '2019')
plt.plot(df_ondernemingen[(df_ondernemingen['Datum'] > pd.Timestamp(2020,1,1)) &(df_ondernemingen['Datum'] < pd.Timestamp(2021,1,1))].set_index('Datum')[['Doorverbindingen_ondernemingen']],marker = ',', label = '2020')
plt.plot(df_ondernemingen[(df_ondernemingen['Datum'] > pd.Timestamp(2021,1,1)) &(df_ondernemingen['Datum'] < pd.Timestamp(2022,1,1))].set_index('Datum')[['Doorverbindingen_ondernemingen']],marker = ',', label = '2021')
plt.plot(df_ondernemingen[(df_ondernemingen['Datum'] > pd.Timestamp(2022,1,1)) &(df_ondernemingen['Datum'] < pd.Timestamp(2023,1,1))].set_index('Datum')[['Doorverbindingen_ondernemingen']],marker = ',', label = '2022')

df_ondernemingen['SMA_calls'] = df_ondernemingen['Doorverbindingen_ondernemingen'].rolling(30).mean()
plt.plot(df_ondernemingen['Datum'], df_ondernemingen['SMA_calls'], color = 'black')

plt.legend()
# seasonality, residual and randomness

In [ ]:
df_ondernemingen['Doorverbindingen_ondernemingen'].mean()

In [ ]:
unieke_jaren = list(df_ondernemingen['Jaar'].unique())
for i, jaren in enumerate(unieke_jaren):
    fig, ax = plt.subplots()
    fig.set_size_inches((12,4))
    ax.set_ylim([0,40000])
    sns.boxplot(x = 'Maand', y = 'Doorverbindingen_ondernemingen', data = df_ondernemingen[df_ondernemingen['Jaar'] == jaren], ax=ax)
    plt.title('Doorverbindingen voor ondernemingen per maand in ' +str(unieke_jaren[i]))
    plt.show()

#### Intermediairs

In [ ]:
df_intermediairs[(df_intermediairs['Datum'] > pd.Timestamp(2019,1,1)) &(df_intermediairs['Datum'] < pd.Timestamp(2020,1,1))]\
                .set_index('Datum')[['Doorverbindingen_intermediairs']]\
                .plot(title="Calls intermediairs 2019",figsize=(12,8))
plt.xticks(rotation = 90)

df_intermediairs[(df_intermediairs['Datum'] > pd.Timestamp(2020,1,1)) &(df_intermediairs['Datum'] < pd.Timestamp(2021,1,1))]\
                .set_index('Datum')[['Doorverbindingen_intermediairs']]\
                .plot(title="Calls intermediairs 2020",figsize=(12,8))
plt.xticks(rotation = 90)

df_intermediairs[(df_intermediairs['Datum'] > pd.Timestamp(2021,1,1)) &(df_intermediairs['Datum'] < pd.Timestamp(2022,1,1))]\
                .set_index('Datum')[['Doorverbindingen_intermediairs']]\
                .plot(title="Calls intermediairs 2021",figsize=(12,8))
plt.xticks(rotation = 90)

df_intermediairs[(df_intermediairs['Datum'] > pd.Timestamp(2022,1,1)) &(df_intermediairs['Datum'] < pd.Timestamp(2023,1,1))]\
                .set_index('Datum')[['Doorverbindingen_intermediairs']]\
                .plot(title="Calls intermediairs 2022",figsize=(12,8))
plt.xticks(rotation = 90)

plt.show()

In [ ]:
df_intermediairs['Doorverbindingen_intermediairs'].mean()

In [ ]:
plt.figure(figsize = (20, 8))
plt.title('Calls intermediairs 2019/01-2022/09')
plt.xlabel('Datum')
plt.ylabel('Aantal calls')
#plt.ylim(0,42000)

plt.plot(df_intermediairs[(df_intermediairs['Datum'] > pd.Timestamp(2019,1,1)) &(df_intermediairs['Datum'] < pd.Timestamp(2020,1,1))].set_index('Datum')[['Doorverbindingen_intermediairs']],marker=',', label= '2019')
plt.plot(df_intermediairs[(df_intermediairs['Datum'] > pd.Timestamp(2020,1,1)) &(df_intermediairs['Datum'] < pd.Timestamp(2021,1,1))].set_index('Datum')[['Doorverbindingen_intermediairs']],marker = ',', label = '2020')
plt.plot(df_intermediairs[(df_intermediairs['Datum'] > pd.Timestamp(2021,1,1)) &(df_intermediairs['Datum'] < pd.Timestamp(2022,1,1))].set_index('Datum')[['Doorverbindingen_intermediairs']],marker = ',', label = '2021')
plt.plot(df_intermediairs[(df_intermediairs['Datum'] > pd.Timestamp(2022,1,1)) &(df_intermediairs['Datum'] < pd.Timestamp(2023,1,1))].set_index('Datum')[['Doorverbindingen_intermediairs']],marker = ',', label = '2022')

df_intermediairs['SMA_calls'] = df_intermediairs['Doorverbindingen_intermediairs'].rolling(30).mean()
plt.plot(df_intermediairs['Datum'], df_intermediairs['SMA_calls'], color = 'black')

plt.legend()

In [ ]:
unieke_jaren = list(df_intermediairs['Jaar'].unique())
for i, jaren in enumerate(unieke_jaren):
    fig, ax = plt.subplots()
    fig.set_size_inches((12,4))
    ax.set_ylim([0,10000])
    sns.boxplot(x = 'Maand', y = 'Doorverbindingen_intermediairs', data = df_intermediairs[df_intermediairs['Jaar'] == jaren], ax=ax)
    plt.title('Doorverbindingen voor intermediairs per maand in ' +str(unieke_jaren[i]))
    plt.show()

#### Toeslagen

In [ ]:
df_toeslagen[(df_toeslagen['Datum'] > pd.Timestamp(2019,1,1)) &(df_toeslagen['Datum'] < pd.Timestamp(2020,1,1))]\
                .set_index('Datum')[['Doorverbindingen_toeslagen']]\
                .plot(title="Calls toeslagen 2019",figsize=(12,8))
plt.xticks(rotation = 90)

df_toeslagen[(df_toeslagen['Datum'] > pd.Timestamp(2020,1,1)) &(df_toeslagen['Datum'] < pd.Timestamp(2021,1,1))]\
                .set_index('Datum')[['Doorverbindingen_toeslagen']]\
                .plot(title="Calls toeslagen 2020",figsize=(12,8))
plt.xticks(rotation = 90)

df_toeslagen[(df_toeslagen['Datum'] > pd.Timestamp(2021,1,1)) &(df_toeslagen['Datum'] < pd.Timestamp(2022,1,1))]\
                .set_index('Datum')[['Doorverbindingen_toeslagen']]\
                .plot(title="Calls toeslagen 2021",figsize=(12,8))
plt.xticks(rotation = 90)

df_toeslagen[(df_toeslagen['Datum'] > pd.Timestamp(2022,1,1)) &(df_toeslagen['Datum'] < pd.Timestamp(2023,1,1))]\
                .set_index('Datum')[['Doorverbindingen_toeslagen']]\
                .plot(title="Calls toeslagen 2022",figsize=(12,8))
plt.xticks(rotation = 90)

plt.show()

In [ ]:
print('mean: ', df_toeslagen['Doorverbindingen_toeslagen'].mean())
print('max: ', df_toeslagen['Doorverbindingen_toeslagen'].max())
print('min: ', df_toeslagen['Doorverbindingen_toeslagen'].min())

In [ ]:
df_toeslagen[df_toeslagen['Doorverbindingen_toeslagen'] == df_toeslagen['Doorverbindingen_toeslagen'].max()]

In [ ]:
df_toeslagen[df_toeslagen['Jaar'] == '2019'][df_toeslagen['Maand'] == 'Dec']

In [ ]:
plt.figure(figsize = (20, 8))
plt.title('Calls toeslagen 2019/01-2022/09')
plt.xlabel('Datum')
plt.ylabel('Aantal calls')
#plt.ylim(0,42000)

plt.plot(df_toeslagen[(df_toeslagen['Datum'] > pd.Timestamp(2019,1,1)) &(df_toeslagen['Datum'] < pd.Timestamp(2020,1,1))].set_index('Datum')[['Doorverbindingen_toeslagen']],marker=',', label= '2019')
plt.plot(df_toeslagen[(df_toeslagen['Datum'] > pd.Timestamp(2020,1,1)) &(df_toeslagen['Datum'] < pd.Timestamp(2021,1,1))].set_index('Datum')[['Doorverbindingen_toeslagen']],marker = ',', label = '2020')
plt.plot(df_toeslagen[(df_toeslagen['Datum'] > pd.Timestamp(2021,1,1)) &(df_toeslagen['Datum'] < pd.Timestamp(2022,1,1))].set_index('Datum')[['Doorverbindingen_toeslagen']],marker = ',', label = '2021')
plt.plot(df_toeslagen[(df_toeslagen['Datum'] > pd.Timestamp(2022,1,1)) &(df_toeslagen['Datum'] < pd.Timestamp(2023,1,1))].set_index('Datum')[['Doorverbindingen_toeslagen']],marker = ',', label = '2022')

df_toeslagen['SMA_calls'] = df_toeslagen['Doorverbindingen_toeslagen'].rolling(30).mean()
plt.plot(df_toeslagen['Datum'], df_toeslagen['SMA_calls'], color = 'black')

plt.legend()

In [ ]:
unieke_jaren = list(df_toeslagen['Jaar'].unique())
for i, jaren in enumerate(unieke_jaren):
    fig, ax = plt.subplots()
    fig.set_size_inches((12,4))
    ax.set_ylim([0,40000])
    sns.boxplot(x = 'Maand', y = 'Doorverbindingen_toeslagen', data = df_toeslagen[df_toeslagen['Jaar'] == jaren], ax=ax)
    plt.title('Doorverbindingen voor toeslagen per maand in ' +str(unieke_jaren[i]))
    plt.show()

#### Auto

In [ ]:
df_auto[(df_auto['Datum'] > pd.Timestamp(2019,1,1)) &(df_auto['Datum'] < pd.Timestamp(2020,1,1))]\
                .set_index('Datum')[['Doorverbindingen_auto']]\
                .plot(title="Calls auto 2019",figsize=(12,8))
plt.xticks(rotation = 90)

df_auto[(df_auto['Datum'] > pd.Timestamp(2020,1,1)) &(df_auto['Datum'] < pd.Timestamp(2021,1,1))]\
                .set_index('Datum')[['Doorverbindingen_auto']]\
                .plot(title="Calls auto 2020",figsize=(12,8))
plt.xticks(rotation = 90)

df_auto[(df_auto['Datum'] > pd.Timestamp(2021,1,1)) &(df_auto['Datum'] < pd.Timestamp(2022,1,1))]\
                .set_index('Datum')[['Doorverbindingen_auto']]\
                .plot(title="Calls auto 2021",figsize=(12,8))
plt.xticks(rotation = 90)

df_auto[(df_auto['Datum'] > pd.Timestamp(2022,1,1)) &(df_auto['Datum'] < pd.Timestamp(2023,1,1))]\
                .set_index('Datum')[['Doorverbindingen_auto']]\
                .plot(title="Calls auto 2022",figsize=(12,8))
plt.xticks(rotation = 90)

plt.show() 

In [ ]:
print('mean: ', df_auto['Doorverbindingen_auto'].mean())
print('max: ', df_auto['Doorverbindingen_auto'].max())
print('min: ', df_auto['Doorverbindingen_auto'].min())

In [ ]:
df_auto[df_auto['Doorverbindingen_auto'] == df_auto['Doorverbindingen_auto'].max()]

In [ ]:
plt.figure(figsize = (20, 8))
plt.title('Calls auto 2019/01-2022/09')
plt.xlabel('Datum')
plt.ylabel('Aantal calls')
#plt.ylim(0,42000)

plt.plot(df_auto[(df_auto['Datum'] > pd.Timestamp(2019,1,1)) &(df_auto['Datum'] < pd.Timestamp(2020,1,1))].set_index('Datum')[['Doorverbindingen_auto']],marker=',', label= '2019')
plt.plot(df_auto[(df_auto['Datum'] > pd.Timestamp(2020,1,1)) &(df_auto['Datum'] < pd.Timestamp(2021,1,1))].set_index('Datum')[['Doorverbindingen_auto']],marker = ',', label = '2020')
plt.plot(df_auto[(df_auto['Datum'] > pd.Timestamp(2021,1,1)) &(df_auto['Datum'] < pd.Timestamp(2022,1,1))].set_index('Datum')[['Doorverbindingen_auto']],marker = ',', label = '2021')
plt.plot(df_auto[(df_auto['Datum'] > pd.Timestamp(2022,1,1)) &(df_auto['Datum'] < pd.Timestamp(2023,1,1))].set_index('Datum')[['Doorverbindingen_auto']],marker = ',', label = '2022')

df_auto['SMA_calls'] = df_auto['Doorverbindingen_auto'].rolling(30).mean()
plt.plot(df_auto['Datum'], df_auto['SMA_calls'], color = 'black')

plt.legend()

In [ ]:
unieke_jaren = list(df_auto['Jaar'].unique())
for i, jaren in enumerate(unieke_jaren):
    fig, ax = plt.subplots()
    fig.set_size_inches((12,4))
    ax.set_ylim([0,10000])
    sns.boxplot(x = 'Maand', y = 'Doorverbindingen_auto', data = df_auto[df_auto['Jaar'] == jaren], ax=ax)
    plt.title('Doorverbindingen voor auto per maand in ' +str(unieke_jaren[i]))
    plt.show()

#### Buitenland

In [ ]:
df_buitenland[(df_buitenland['Datum'] > pd.Timestamp(2019,1,1)) &(df_buitenland['Datum'] < pd.Timestamp(2020,1,1))]\
                .set_index('Datum')[['Doorverbindingen_buitenland']]\
                .plot(title="Calls buitenland 2019",figsize=(12,8))
plt.xticks(rotation = 90)

df_buitenland[(df_buitenland['Datum'] > pd.Timestamp(2020,1,1)) &(df_buitenland['Datum'] < pd.Timestamp(2021,1,1))]\
                .set_index('Datum')[['Doorverbindingen_buitenland']]\
                .plot(title="Calls buitenland 2020",figsize=(12,8))
plt.xticks(rotation = 90)

df_buitenland[(df_buitenland['Datum'] > pd.Timestamp(2021,1,1)) &(df_buitenland['Datum'] < pd.Timestamp(2022,1,1))]\
                .set_index('Datum')[['Doorverbindingen_buitenland']]\
                .plot(title="Calls buitenland 2021",figsize=(12,8))
plt.xticks(rotation = 90)

df_buitenland[(df_buitenland['Datum'] > pd.Timestamp(2022,1,1)) &(df_buitenland['Datum'] < pd.Timestamp(2023,1,1))]\
                .set_index('Datum')[['Doorverbindingen_buitenland']]\
                .plot(title="Calls buitenland 2022",figsize=(12,8))
plt.xticks(rotation = 90)

plt.show() 

In [ ]:
print('mean: ', df_buitenland['Doorverbindingen_buitenland'].mean())
print('max: ', df_buitenland['Doorverbindingen_buitenland'].max())
print('min: ', df_buitenland['Doorverbindingen_buitenland'].min())

In [ ]:
plt.figure(figsize = (20, 8))
plt.title('Calls buitenland 2019/01-2022/09')
plt.xlabel('Datum')
plt.ylabel('Aantal calls')
#plt.ylim(0,42000)

plt.plot(df_buitenland[(df_buitenland['Datum'] > pd.Timestamp(2019,1,1)) &(df_buitenland['Datum'] < pd.Timestamp(2020,1,1))].set_index('Datum')[['Doorverbindingen_buitenland']],marker=',', label= '2019')
plt.plot(df_buitenland[(df_buitenland['Datum'] > pd.Timestamp(2020,1,1)) &(df_buitenland['Datum'] < pd.Timestamp(2021,1,1))].set_index('Datum')[['Doorverbindingen_buitenland']],marker = ',', label = '2020')
plt.plot(df_buitenland[(df_buitenland['Datum'] > pd.Timestamp(2021,1,1)) &(df_buitenland['Datum'] < pd.Timestamp(2022,1,1))].set_index('Datum')[['Doorverbindingen_buitenland']],marker = ',', label = '2021')
plt.plot(df_buitenland[(df_buitenland['Datum'] > pd.Timestamp(2022,1,1)) &(df_buitenland['Datum'] < pd.Timestamp(2023,1,1))].set_index('Datum')[['Doorverbindingen_buitenland']],marker = ',', label = '2022')

df_buitenland['SMA_calls'] = df_buitenland['Doorverbindingen_buitenland'].rolling(30).mean()
plt.plot(df_buitenland['Datum'], df_buitenland['SMA_calls'], color = 'black')

plt.legend()

In [ ]:
unieke_jaren = list(df_buitenland['Jaar'].unique())
for i, jaren in enumerate(unieke_jaren):
    fig, ax = plt.subplots()
    fig.set_size_inches((12,4))
    ax.set_ylim([0,10000])
    sns.boxplot(x = 'Maand', y = 'Doorverbindingen_buitenland', data = df_buitenland[df_buitenland['Jaar'] == jaren], ax=ax)
    plt.title('Doorverbindingen voor buitenland per maand in ' +str(unieke_jaren[i]))
    plt.show()

#### Overig

In [ ]:
df_overig[(df_overig['Datum'] > pd.Timestamp(2019,1,1)) &(df_overig['Datum'] < pd.Timestamp(2020,1,1))]\
                .set_index('Datum')[['Doorverbindingen_overig']]\
                .plot(title="Calls overig 2019",figsize=(12,8))
plt.xticks(rotation = 90)

df_overig[(df_overig['Datum'] > pd.Timestamp(2020,1,1)) &(df_overig['Datum'] < pd.Timestamp(2021,1,1))]\
                .set_index('Datum')[['Doorverbindingen_overig']]\
                .plot(title="Calls overig 2020",figsize=(12,8))
plt.xticks(rotation = 90)

df_overig[(df_overig['Datum'] > pd.Timestamp(2021,1,1)) &(df_overig['Datum'] < pd.Timestamp(2022,1,1))]\
                .set_index('Datum')[['Doorverbindingen_overig']]\
                .plot(title="Calls overig 2021",figsize=(12,8))
plt.xticks(rotation = 90)

df_overig[(df_overig['Datum'] > pd.Timestamp(2022,1,1)) &(df_overig['Datum'] < pd.Timestamp(2023,1,1))]\
                .set_index('Datum')[['Doorverbindingen_overig']]\
                .plot(title="Calls overig 2022",figsize=(12,8))
plt.xticks(rotation = 90)

plt.show() 

In [ ]:
plt.figure(figsize = (20, 8))
plt.title('Calls overig 2019/01-2022/09')
plt.xlabel('Datum')
plt.ylabel('Aantal calls')
#plt.ylim(0,42000)

plt.plot(df_overig[(df_overig['Datum'] > pd.Timestamp(2019,1,1)) &(df_overig['Datum'] < pd.Timestamp(2020,1,1))].set_index('Datum')[['Doorverbindingen_overig']],marker=',', label= '2019')
plt.plot(df_overig[(df_overig['Datum'] > pd.Timestamp(2020,1,1)) &(df_overig['Datum'] < pd.Timestamp(2021,1,1))].set_index('Datum')[['Doorverbindingen_overig']],marker = ',', label = '2020')
plt.plot(df_overig[(df_overig['Datum'] > pd.Timestamp(2021,1,1)) &(df_overig['Datum'] < pd.Timestamp(2022,1,1))].set_index('Datum')[['Doorverbindingen_overig']],marker = ',', label = '2021')
plt.plot(df_overig[(df_overig['Datum'] > pd.Timestamp(2022,1,1)) &(df_overig['Datum'] < pd.Timestamp(2023,1,1))].set_index('Datum')[['Doorverbindingen_overig']],marker = ',', label = '2022')

df_overig['SMA_calls'] = df_overig['Doorverbindingen_overig'].rolling(30).mean()
plt.plot(df_overig['Datum'], df_overig['SMA_calls'], color = 'black')

plt.legend()

In [ ]:
df_overig['Doorverbindingen_overig'].mean()

In [ ]:
unieke_jaren = list(df_overig['Jaar'].unique())
for i, jaren in enumerate(unieke_jaren):
    fig, ax = plt.subplots()
    fig.set_size_inches((12,4))
    ax.set_ylim([0,10000])
    sns.boxplot(x = 'Maand', y = 'Doorverbindingen_overig', data = df_overig[df_overig['Jaar'] == jaren], ax=ax)
    plt.title('Doorverbindingen voor overig per maand in ' +str(unieke_jaren[i]))
    plt.show()

### Trend van ophangers t.o.v. wachttijd

In [ ]:
df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2019,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2020,1,1))]\
                .set_index('Datum')[['Wachtrij_opgehangen_totaal', 'Gemiddelde_wachttijd_totaal']]\
                .plot(title="Ophangers vs wachttijd totaal 2019",figsize=(12,8))
plt.xticks(rotation = 90)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2020,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2021,1,1))]\
                .set_index('Datum')[['Wachtrij_opgehangen_totaal', 'Gemiddelde_wachttijd_totaal']]\
                .plot(title="Ophangers vs wachttijd totaal 2020",figsize=(12,8))
plt.xticks(rotation = 90)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2021,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2022,1,1))]\
                .set_index('Datum')[['Wachtrij_opgehangen_totaal', 'Gemiddelde_wachttijd_totaal']]\
                .plot(title="Ophangers vs wachttijd totaal 2021",figsize=(12,8))
plt.xticks(rotation = 90)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2022,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2023,1,1))]\
                .set_index('Datum')[['Wachtrij_opgehangen_totaal', 'Gemiddelde_wachttijd_totaal']]\
                .plot(title="Ophangers vs wachttijd totaal 2022",figsize=(12,8))
plt.xticks(rotation = 90)

plt.show()

### Trend van inkomende calls t.o.v. ophangers

In [ ]:
df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2019,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2020,1,1))]\
                .set_index('Datum')[['Doorgeleid naar keuzemenu', 'Wachtrij_opgehangen_totaal']]\
                .plot(title="Calls vs ophangers 2019",figsize=(12,8))
plt.xticks(rotation = 90)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2020,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2021,1,1))]\
                .set_index('Datum')[['Doorgeleid naar keuzemenu', 'Wachtrij_opgehangen_totaal']]\
                .plot(title="Calls vs ophangers 2020",figsize=(12,8))
plt.xticks(rotation = 90)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2021,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2022,1,1))]\
                .set_index('Datum')[['Doorgeleid naar keuzemenu', 'Wachtrij_opgehangen_totaal']]\
                .plot(title="Calls vs ophangers 2021",figsize=(12,8))
plt.xticks(rotation = 90)

df_particulieren[(df_particulieren['Datum'] > pd.Timestamp(2022,1,1)) &(df_particulieren['Datum'] < pd.Timestamp(2023,1,1))]\
                .set_index('Datum')[['Doorgeleid naar keuzemenu', 'Wachtrij_opgehangen_totaal']]\
                .plot(title="Calls vs ophangers 2022",figsize=(12,8))
plt.xticks(rotation = 90)

plt.show()

In [ ]:
df_particulieren.columns

### Vergelijking trends in geholpen bellers tussen categorieen

In [ ]:
plt.figure(figsize = (20, 8))
plt.title('Calls per thema 2019/01-2022/09')
plt.xlabel('Datum')
plt.ylabel('Aantal calls') 

#plt.plot(df_particulieren['Datum'], df_particulieren['Geholpen_totaal'], marker=',', color='black', label= 'Particulieren')
plt.plot(df_particulieren['Datum'], df_particulieren['Geholpen_particulieren'], color='green', label= 'Particulieren')
plt.plot(df_ondernemingen['Datum'], df_ondernemingen['Geholpen_ondernemingen'], color = 'red',label = 'Ondernemingen')
plt.plot(df_auto['Datum'], df_auto['Geholpen_auto'], color='blue', label= 'Auto')
plt.plot(df_intermediairs['Datum'], df_intermediairs['Geholpen_intermediairs'], color = 'yellow',label = 'Intermediairs')
plt.plot(df_buitenland['Datum'], df_buitenland['Geholpen_buitenland'], color = 'orange',label = 'Buitenland')
plt.plot(df_toeslagen['Datum'], df_toeslagen['Geholpen_toeslagen'], color='purple', label= 'Toeslagen')
plt.plot(df_overig['Datum'], df_overig['Geholpen_overig'], color = 'brown',label = 'Overig')
plt.legend()

In [ ]:
plt.figure(figsize = (20, 8))
plt.title('Calls per thema 2019/01-2022/09')
plt.xlabel('Datum')
plt.ylabel('Aantal calls') 

#plt.plot(df_particulieren['Datum'], df_particulieren['Geholpen_totaal'], marker=',', color='black', label= 'Particulieren')
plt.plot(df_particulieren['Datum'], df_particulieren['Geholpen_particulieren'], label= 'Particulieren')
plt.plot(df_ondernemingen['Datum'], df_ondernemingen['Geholpen_ondernemingen'], label = 'Ondernemingen')
plt.plot(df_auto['Datum'], df_auto['Geholpen_auto'], label= 'Auto')
plt.plot(df_intermediairs['Datum'], df_intermediairs['Geholpen_intermediairs'], label = 'Intermediairs')
plt.plot(df_buitenland['Datum'], df_buitenland['Geholpen_buitenland'], label = 'Buitenland')
plt.plot(df_toeslagen['Datum'], df_toeslagen['Geholpen_toeslagen'], label= 'Toeslagen')
plt.plot(df_overig['Datum'], df_overig['Geholpen_overig'], label = 'Overig')
plt.legend()

### Vergelijking trends in bellers tussen categorieen

In [ ]:
plt.figure(figsize = (20, 8))
plt.plot(df_particulieren['Datum'], df_particulieren['Doorverbindingen_particulieren'], color='green', label= 'Particulieren')
plt.plot(df_ondernemingen['Datum'], df_ondernemingen['Doorverbindingen_ondernemingen'], color = 'red',label = 'Ondernemingen')
plt.plot(df_auto['Datum'], df_auto['Doorverbindingen_auto'], color='indianred', label= 'blue')
plt.plot(df_intermediairs['Datum'], df_intermediairs['Doorverbindingen_intermediairs'], color = 'yellow',label = 'Intermediairs')
plt.plot(df_buitenland['Datum'], df_buitenland['Doorverbindingen_buitenland'], color = 'orange',label = 'Buitenland')
plt.plot(df_toeslagen['Datum'], df_toeslagen['Doorverbindingen_toeslagen'], color='purple', label= 'Toeslagen')
plt.plot(df_overig['Datum'], df_overig['Doorverbindingen_overig'], color = 'brown',label = 'Overig')
plt.legend()

In [ ]:
plt.figure(figsize = (20, 8))
plt.plot(df_particulieren['Datum'], df_particulieren['Doorverbindingen_particulieren'], label= 'Particulieren')
plt.plot(df_ondernemingen['Datum'], df_ondernemingen['Doorverbindingen_ondernemingen'], label = 'Ondernemingen')
plt.plot(df_auto['Datum'], df_auto['Doorverbindingen_auto'], label= 'blue')
plt.plot(df_intermediairs['Datum'], df_intermediairs['Doorverbindingen_intermediairs'], label = 'Intermediairs')
plt.plot(df_buitenland['Datum'], df_buitenland['Doorverbindingen_buitenland'], label = 'Buitenland')
plt.plot(df_toeslagen['Datum'], df_toeslagen['Doorverbindingen_toeslagen'], label= 'Toeslagen')
plt.plot(df_overig['Datum'], df_overig['Doorverbindingen_overig'], label = 'Overig')
plt.legend()

### Moving average of calls

In [ ]:
df_particulieren['SMA_calls'] = df_particulieren['Doorverbindingen_particulieren'].rolling(30).mean()#.dropna()

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(df_particulieren['Datum'], df_particulieren['Doorverbindingen_particulieren'])
plt.plot(df_particulieren['Datum'], df_particulieren['SMA_calls'])
# added moving averages to call plots 

### Complexity score

#### Correlations in data with complexity score

In [ ]:
df_totaal['ComplexityScore_ondernemingen']

In [ ]:
df_ranked_calls = df_totaal[[('Doorverbindingen', 'AUTO'),
                            ('Doorverbindingen', 'BUITENLAND'),
                            ('Doorverbindingen', 'INTERMEDIAIRS'),
                            ('Doorverbindingen', 'ONDERNEMINGEN'),
                            ('Doorverbindingen', 'OVERIG'),
                            ('Doorverbindingen', 'PARTICULIEREN'),
                            ('Doorverbindingen', 'TOESLAGEN'),
                            ]].rank(axis =1, method = 'average', ascending = False)

df_ranked_calls.plot(figsize = (8,6), title = 'Rank per thema per dag', xlabel = 'Tijd in dagen', ylabel = 'Rank')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
df_rank = pd.DataFrame({'Rank':df_totaal[[('Doorverbindingen', 'AUTO'),
              ('Doorverbindingen', 'BUITENLAND'),
           ('Doorverbindingen', 'INTERMEDIAIRS'),
           ('Doorverbindingen', 'ONDERNEMINGEN'),
                  ('Doorverbindingen', 'OVERIG'),
           ('Doorverbindingen', 'PARTICULIEREN'),
               ('Doorverbindingen', 'TOESLAGEN'),
          ]].rank(axis =1, method = 'average', ascending = False).mean().sort_values(), 'Absolute rank': [1,2,3,4,5,6,7], 'Score':[2,6,1,4,5,7,3]})